## Loading in packages

In [1]:
##### LOADING IN PACKAGES #--------------------------------------------------------------
import s3fs; import xarray as xr; import numpy as np
import pandas as pd; 
import dask.array as da
import ocetrac

import matplotlib.pyplot as plt; import cartopy.crs as ccrs

import warnings; import expectexception
warnings.filterwarnings('ignore')

import netCDF4 as nc; import datetime as dt
import scipy

import intake; import pprint
# Allow multiple lines per cell to be displayed without print (default is just last line)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Enable more explicit control of DataFrame display (e.g., to omit annoying line numbers)
from IPython.display import HTML

## Loading CESM LE dataset

In [2]:
# Open original collection description file #----------------------------------------------
cat_url_orig = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
coll_orig = intake.open_esm_datastore(cat_url_orig)

In [3]:
subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical')
member_id_list = subset.df.member_id.unique()
print(member_id_list)

['r1i1001p1f1' 'r2i1021p1f1' 'r3i1041p1f1' 'r4i1061p1f1' 'r5i1081p1f1'
 'r6i1101p1f1' 'r7i1121p1f1' 'r8i1141p1f1' 'r9i1161p1f1' 'r10i1181p1f1'
 'r1i1231p1f1' 'r2i1231p1f1' 'r3i1231p1f1' 'r4i1231p1f1' 'r5i1231p1f1'
 'r6i1231p1f1' 'r7i1231p1f1' 'r8i1231p1f1' 'r9i1231p1f1' 'r10i1231p1f1'
 'r1i1251p1f1' 'r2i1251p1f1' 'r3i1251p1f1' 'r4i1251p1f1' 'r5i1251p1f1'
 'r6i1251p1f1' 'r7i1251p1f1' 'r8i1251p1f1' 'r9i1251p1f1' 'r10i1251p1f1'
 'r1i1281p1f1' 'r2i1281p1f1' 'r3i1281p1f1' 'r4i1281p1f1' 'r5i1281p1f1'
 'r6i1281p1f1' 'r7i1281p1f1' 'r8i1281p1f1' 'r9i1281p1f1' 'r10i1281p1f1'
 'r1i1301p1f1' 'r2i1301p1f1' 'r3i1301p1f1' 'r4i1301p1f1' 'r5i1301p1f1'
 'r6i1301p1f1' 'r7i1301p1f1' 'r8i1301p1f1' 'r9i1301p1f1' 'r10i1301p1f1'
 'r1i1011p1f2' 'r2i1031p1f2' 'r3i1051p1f2' 'r4i1071p1f2' 'r5i1091p1f2'
 'r6i1111p1f2' 'r7i1131p1f2' 'r8i1151p1f2' 'r9i1171p1f2' 'r10i1191p1f2'
 'r11i1231p1f2' 'r12i1231p1f2' 'r13i1231p1f2' 'r14i1231p1f2'
 'r15i1231p1f2' 'r16i1231p1f2' 'r17i1231p1f2' 'r18i1231p1f2'
 'r19i1231p1f2' 'r20

In [4]:
##### one at a time
# subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical',member_id= 'r1i1011p1f2')
# dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
# dsets
# ds = dsets['atm.historical.cam.h0.smbb.SST']

In [ ]:
for i in range(50,len(member_id_list)):
# for i in range(5):
    subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical',member_id= str(member_id_list[i]))
    dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
    # ds = dsets['atm.historical.cam.h0.cmip6.SST'] # before 50
    ds = dsets['atm.historical.cam.h0.smbb.SST'] # after 50 # Ask Liz
    SST = ds.SST.isel(member_id=0)
    SST.load()
    
    ###### DETRENDING
    # last 40 years (satellite period)
    dyr = SST.time.dt.year + (SST.time.dt.month-0.5)/12
    dyr = dyr[-481:] # can remove this line if you want to detrend across the entire period
    # Our 6 coefficient model is composed of the mean, trend, annual sine and cosine harmonics, & semi-annual sine and cosine harmonics
    model = np.array([np.ones(len(dyr))] + [dyr-np.mean(dyr)] + [np.sin(2*np.pi*dyr)] + [np.cos(2*np.pi*dyr)] + [np.sin(4*np.pi*dyr)] + [np.cos(4*np.pi*dyr)])
    # Take the pseudo-inverse of model to 'solve' least-squares problem
    pmodel = np.linalg.pinv(model)
    model_da = xr.DataArray(model.T, dims=['time','coeff'], coords={'time':SST.time.values[-481:], 'coeff':np.arange(1,7,1)}) 
    pmodel_da = xr.DataArray(pmodel.T, dims=['coeff','time'], coords={'coeff':np.arange(1,7,1), 'time':SST.time.values[-481:]})
    # resulting coefficients of the model
    sst_mod = xr.DataArray(pmodel_da.dot(SST), dims=['coeff','lat','lon'], coords={'coeff':np.arange(1,7,1), 'lat':SST.lat.values, 'lon':SST.lon.values})
    # Construct mean, trend, and seasonal cycle
    mean = model_da[:,0].dot(sst_mod[0,:,:])
    trend = model_da[:,1].dot(sst_mod[1,:,:])
    seas = model_da[:,2:].dot(sst_mod[2:,:,:])
    # compute anomalies by removing all  the model coefficients 
    ssta_notrend = SST-model_da.dot(sst_mod) #this is anomalies
    detrended = ssta_notrend
    detrended.to_netcdf('/glade/work/cassiacai/'+str(member_id_list[i])+'_detrended.nc')
    
    ###### THRESHOLD and FEATURES
    if detrended.chunks:
        detrended = detrended.chunk({'time': -1})
    threshold = detrended.groupby('time.month').quantile(0.9,dim=('time')) 
    features_ssta = detrended.where(detrended.groupby('time.month')>=threshold, other=np.nan)
    features_ssta = features_ssta[:,:,:].load()
    ##### MASKING
    full_mask_land = features_ssta
    full_masked = full_mask_land.where(full_mask_land != 0)
    binary_out_afterlandmask=np.isfinite(full_masked)
    
    newmask = np.isfinite(ds.SST[0,:,:,:][-481:])
    
    Tracker = ocetrac.Tracker(binary_out_afterlandmask[:,:,:], newmask, radius=3, min_size_quartile=0.75, timedim = 'time', xdim = 'lon', ydim='lat', positive=True)
    blobs = Tracker.track()
    blobs.attrs
    mo = Tracker._morphological_operations()
    blobs.to_netcdf('/glade/work/cassiacai/'+str(member_id_list[i])+'_rad3_blobs.nc')


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


<xarray.DataArray 'SST' (time: 1980, lat: 192, lon: 288)>
array([[[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.3471 , 271.34705, 271.34705, ..., 271.3471 , 271.3471 ,
         271.3471 ],
        [271.35165, 271.35168, 271.3517 , ..., 271.3516 , 271.35162,
         271.35162],
        [271.35437, 271.35437, 271.35437, ..., 271.35434, 271.35437,
         271.35437]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
...
        [271.40085, 271.40082, 271.40082, ..., 271.4009 , 271.4009 ,
         271.4009 ],
        [271.40445, 271.40445, 271.40445, ..., 271.40445, 271.40442,
         271.40445],
        [271.4066 , 271.4066 , 271.4066 , ..., 271.40662, 271.4066 ,
         271.4066 ]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.3863 , 271.38617, 271.3861 , ..., 271.38657, 271.3865 ,
         271.38644],
        [271.39032, 271.39032, 271.3903 , ..., 271.39044, 271.39038,
         271.39035],
        [271.39294, 271.39294, 271.3929 , ..., 271.39294, 271.39294,
         271.39294]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r1i1011p1f2'
Attributes:
    units:         K
    long_name:     sea surface temperature
    cell_methods:  time: mean

minimum area: 327.0
inital objects identified 	 5480
final objects tracked 	 330


{'inital objects identified': 5480,
 'final objects tracked': 330,
 'radius': 3,
 'size quantile threshold': 0.75,
 'min area': 327.0,
 'percent area reject': 0.27498956405891495,
 'percent area accept': 0.725010435941085}


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


<xarray.DataArray 'SST' (time: 1980, lat: 192, lon: 288)>
array([[[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.33505, 271.33505, 271.33508, ..., 271.33502, 271.33502,
         271.33505],
        [271.33954, 271.33957, 271.3396 , ..., 271.33954, 271.33954,
         271.33954],
        [271.34262, 271.3426 , 271.3426 , ..., 271.34262, 271.34262,
         271.34262]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
...
        [271.391  , 271.39105, 271.3911 , ..., 271.39087, 271.3909 ,
         271.39096],
        [271.39648, 271.39655, 271.39658, ..., 271.39642, 271.39642,
         271.39645],
        [271.3999 , 271.3999 , 271.3999 , ..., 271.3999 , 271.3999 ,
         271.3999 ]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.38144, 271.38147, 271.3815 , ..., 271.38144, 271.38144,
         271.38147],
        [271.38712, 271.38715, 271.38715, ..., 271.3871 , 271.3871 ,
         271.38712],
        [271.39096, 271.39096, 271.39096, ..., 271.39096, 271.39096,
         271.39096]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r2i1031p1f2'
Attributes:
    units:         K
    long_name:     sea surface temperature
    cell_methods:  time: mean